In [ ]:
 # Nur einmal pro Session herunterladen
 !pip install open-webui

# Beyond Correlation: Why Likert-Scale Calibration Challenges LLM-as-Participant Designs.



# Code structure: #

In the **first block** of the code, some libraries (such as openai) are imported and the own API-key of OpenAI is defined.

In [ ]:
import os
from openai import OpenAI

os.environ["OPENAI_API_KEY"] = "" # insert here OpenAI API key


This is a test, it works! How can I assist you further?


The **second block** of the code contains the prompt engineering functions that we will use later. You will find zero and few shot prompting as example functions. However, only the zero-shot function is applied in the next steps, since few-shot prompting performed poorely in first pilot studies.

In [ ]:
import openai
from openai import OpenAI

# Funktion zur Durchführung von Null-Shot-Prompting
def zero_shot_prompting(task, prompt):
    """
    Führt Null-Shot-Prompting durch, indem ein Prompt an das Sprachmodell gesendet wird,
    ohne dass vorherige Beispiele gegeben werden.

    Args:
    task (str): Die Aufgabe, die durchgeführt werden soll.
    prompt (str): Der Text, der als Eingabe an das Sprachmodell gesendet wird.

    Returns:
    str: Die Antwort des Sprachmodells.
    """
    # Erstellen der Chat-Nachricht und Senden an das Sprachmodell
    response = client.chat.completions.create(
        model="o4-mini",
        messages=[
            {"role": "system", "content": "Tu es un(e) assistant(e)"}, # HIER ÄNDERN, WENN GEWÜNSCHT
            {"role": "user", "content": prompt} # Benutzernachricht mit dem eigentlichen Prompt
        ]
    )
    # Rückgabe des Inhalts der ersten Antwortnachricht des Modells
    return response.choices[0].message.content



# Funktion zur Durchführung von Few-Shot-Prompting
def few_shot_prompting(task, examples, prompt):
    """
    Führt Few-Shot-Prompting durch, indem einige Beispiele gegeben werden,
    bevor der eigentliche Prompt an das Sprachmodell gesendet wird.

    Args:
    task (str): Die Aufgabe, die durchgeführt werden soll.
    examples (list of dict): Eine Liste von Beispielen, wobei jedes Beispiel ein Wörterbuch mit 'input' und 'output' enthält.
    prompt (str): Der Text, der als Eingabe an das Sprachmodell gesendet wird.

    Returns:
    str: Die Antwort des Sprachmodells.
    """
    # Initialisieren der Nachrichtenliste mit einer Systemnachricht
    messages = [{"role": "system", "content": "Tu es un(e) assistant(e)"}] # HIER ÄNDERN, WENN GEWÜNSCHT

    # Hinzufügen der Beispiele zu den Nachrichten
    for example in examples:
        messages.append({"role": "user", "content": example['input']})
        messages.append({"role": "assistant", "content": example['output']})

    # Hinzufügen des eigentlichen Prompts zur Nachrichtenliste
    messages.append({"role": "user", "content": prompt})

    # Erstellen der Chat-Nachricht und Senden an das Sprachmodell
    response = client.chat.completions.create(
        model="o4-mini",
        messages=messages
    )
    # Rückgabe des Inhalts der ersten Antwortnachricht des Modells
    return response.choices[0].message.content


In the **third block** of the code you find code so that you can read in your uploaded data. This will be helpful in presenting the data to the LLM during prompting.

In [ ]:
import pandas as pd
import random
import requests
from collections import Counter
import re

In [ ]:
# Load data
daten = 'Barbosa&Cat_2019.csv'  # Adjust the filename as needed
df = pd.read_csv(daten)

# Extract specific columns and pack them into a dictionary
columns_to_extract = ['French Sentences', 'syntactic position', 'trace position', 'status', 'gold_ratings']
selected_data = df[columns_to_extract].astype(str)
stimuli_dict = selected_data.to_dict(orient="index")

# Create list of items
all_items = [entry['French Sentences'] for entry in stimuli_dict.values()]
random.shuffle(all_items)
print("The materials for the study are:")
print(all_items)

The materials for the study are:
['a. Ursule explique à Joseph comment elle compte lui faire parvenir des messages en lançant des bouteilles à la mer. Intrigué, Joseph demande : b. Comment veux-tu que je te réponde ?', 'a. La maman de Zéphir montre à une amie un vélo rouge dans la vitrine. Elle lui dit : b. Voilà le vélo que depuis toujours il rêve d’essayer.', 'a. Zoé est impatiente de rencontrer le patron de son mari. Elle dit : b. Je me demande quand, ton patron, il va nous inviter.', 'a. Depuis qu’elle est pensionnée, la sœur d’Armande voyage sans arrêt. Armande, amusée, en parle avec une amie. Elle dit : b. Je me demande où, cette semaine, elle a bien pu aller.', 'a. Bruno est figurant dans un film à petit budget. Il raconte à son ami Nicolas : b. On jouait dans un film que, dans quelques années, personne ne regardera plus.', 'a. Pierre se vante à son ami de ses exploits et de ceux de ses frères. Il déclare : b. J’ai vu la forêt où, le loup, ils l’ont traqué.', 'a. Flore a surpris

In the **fourth block** we can apply the prompt engineering functions of block 2. We will formulate our prompts in this cell. In this block, we can test different prompting strategies. We prompt the model as many times as we have (or want to have) particpants tu simulate.

In [ ]:
# -----------------------------
# 1. API-Daten
# -----------------------------
API_URL = # insert your API url
API_KEY = # insert your API key

def query_model(model_name, prompt_text, temperature=0.3, max_tokens=200):
    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt_text}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    response = requests.post(API_URL, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]


# -----------------------------
# 2. Stimuli sind schon geladen
# -----------------------------
# Aus dem vorherigen Block:
# all_items = [...]
stimuli = all_items   # HIER nach Bedarf ändern: Hier werden erstmal zum Probieren nur 2 Stimuli gezeigt. Wenn das Prompt steht, bitte ganze Klammer wegmachen!

print("Anzahl der Stimuli:", len(stimuli))


# -----------------------------
# 3. Anzahl der simulierten Probanden
# -----------------------------
n_probanden = 60          # HIER nach Bedarf ändern
model = "mistral-small:24b"   # HIER nach Bedarf ändern

simulation_results = []   # alle Antworten werden hier gespeichert


# -----------------------------
# 4. Simulation starten
# -----------------------------
print("\nSimulation startet...\n")

for proband in range(1, n_probanden + 1):
    print(f"--- Simulierter Proband {proband} ---")

    for idx, stimulus in enumerate(stimuli):
        # Prompt ändern
        prompt = f"""
        Vous participez à une étude. Vous êtes de langue maternelle française. Tâche : Vous allez maintenant voir deux phrases. La phrase a constitue le contexte, la phrase b est la phrase que vous devez juger, pas la phrase a. Vous devez exprimer votre jugement seulement sur la phrase b en utilisant le schéma suivant : a. Je pourrais dire cela. b. Je pourrais dire cela, mais dans un autre contexte. c. Je ne pourrais pas dire cela, mais je connais des gens qui pourraient le faire. d. Personne ne dirait cela. e. Je ne sais pas. Vous donnez seulement la lettre, pas la justification. Penser : Vous lisez le contenu (phrase a), puis vous vous concentrez sur la phrase b. Vous la lisez attentivement et vous vous demandez si cette formulation est acceptable. Vous vous posez la question : Est-ce que moi, ou quelqu’un d’autre, pourrais dire cela ? Votre réponse vous aide à choisir la lettre qui exprimera votre jugement dans la tâche. Voici les deux phrases: “{stimulus}” Réponse:

        """

        antwort = query_model(model, prompt)

        simulation_results.append({
            "proband": proband,
            "stimulus_index": idx,         # An welcher Position der Stimulus währen der Studie vorkam
            "stimulus_text": stimulus,
            "antwort": antwort
        })

        print(f"Antwort zu Item: '{stimulus}': {antwort}")


print("\nSimulation abgeschlossen!")



Anzahl der Stimuli: 66

Simulation startet...

--- Simulierter Proband 1 ---
Antwort zu Item: 'a. Ursule explique à Joseph comment elle compte lui faire parvenir des messages en lançant des bouteilles à la mer. Intrigué, Joseph demande : b. Comment veux-tu que je te réponde ?': b
Antwort zu Item: 'a. La maman de Zéphir montre à une amie un vélo rouge dans la vitrine. Elle lui dit : b. Voilà le vélo que depuis toujours il rêve d’essayer.': b
Antwort zu Item: 'a. Zoé est impatiente de rencontrer le patron de son mari. Elle dit : b. Je me demande quand, ton patron, il va nous inviter.': b
Antwort zu Item: 'a. Depuis qu’elle est pensionnée, la sœur d’Armande voyage sans arrêt. Armande, amusée, en parle avec une amie. Elle dit : b. Je me demande où, cette semaine, elle a bien pu aller.': b
Antwort zu Item: 'a. Bruno est figurant dans un film à petit budget. Il raconte à son ami Nicolas : b. On jouait dans un film que, dans quelques années, personne ne regardera plus.': b
Antwort zu Item: 'a

In the **fifth block** we save the results from all test subjects in an Excel and csv file. These files subsequently store the answers of all LLMs-participants.

In [ ]:
# 1. Ursprüngliche Stimuli-Datei laden
df = pd.read_csv("Barbosa&Cat_2019.csv")
df = df.rename(columns=lambda c: c.strip())

# 2. simulation_results → DataFrame
sim_df = pd.DataFrame(simulation_results)

# Spalten passend umbenennen
sim_df = sim_df.rename(columns={
    "proband": "Informant",
    "stimulus_index": "Index",
    "stimulus_text": "sentences",
    "antwort": "models_answer"
})

# 3. Merge über target_sent (df) und sentences (sim_df)
merged = df.merge(
    sim_df,
    left_on="French Sentences",
    right_on="sentences",
    how="left"
)

# 4. Überflüssige Spalte entfernen
merged = merged.drop(columns=["sentences"])

# 5. Speichern
merged.to_csv("60_Barbosa&DeCat_mistral_CoT_t0.3.csv", index=False)
#merged.to_excel("Lombard_stimuli_mit_Modell_Antworten.xlsx", index=False)

print("Neue Datei erfolgreich erstellt!")

Neue Datei erfolgreich erstellt!
